# Doc2Vecの実行条件調査（part3）

- ラベル毎にユニークな教師データを設定（このためテストデータから学習セットの大半が失われます）


- 品詞を落とさないようにする


- パラメータを一部変更（DBOWを使用／feature数を調整／反復回数を調整）


accuracy=0.965 となりました。

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

## (2) Doc2Vecの動作確認

### (2-1) コーパス生成

コーパス（単語が半角スペースで区切られた文字列）生成時、一部の品詞を落とすようにします。

（＝learning.core.nlang.Nlang クラスの仕様に従います）

In [2]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 9  # bot_id = 9はセプテーニ
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}

learning_parameter = LearningParameter(attr)

In [3]:
_datasource = Datasource(type='csv')
learning_training_messages = _datasource.learning_training_messages(_bot_id)
questions = np.array(learning_training_messages['question'])
answer_ids = np.array(learning_training_messages['answer_id'])

2017/05/18 PM 01:46:52 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/18 PM 01:46:52 ['./fixtures/question_answers/toyotsu_human.csv']


In [4]:
import MeCab
import mojimoji

class Nlang_naive:
    @classmethod
    def split(self, text):
        tagger = MeCab.Tagger("-u learning/dict/custom.dic")
        tagger.parse('')  # node.surfaceを取得出来るようにするため、空文字をparseする(Python3のバグの模様)
        node = tagger.parseToNode(text)
        word_list = []
        while node:
            features = node.feature.split(",")
            pos = features[0]
            if pos in ["BOS/EOS", "記号"]:
                node = node.next
                continue

            #print(features)
            lemma = node.feature.split(",")[6]

            if lemma == "*":
                lemma = node.surface  #.decode("utf-8")
                
            word_list.append(mojimoji.han_to_zen(lemma))
            node = node.next
        return " ".join(word_list)

    @classmethod
    def batch_split(self, texts):
        splited_texts = []
        for text in texts:
            splited_texts.append(self.split(text))
        return splited_texts

In [5]:
from learning.core.nlang import Nlang

_sentences = np.array(questions)
_separated_sentences = Nlang_naive.batch_split(_sentences)

#### ラベル毎の教師データ数を制限します。

In [6]:
answer_id_count = {}

selected_answer_ids = []
selected_questions = []
selected_separated_sentences = []

sample_limit_count = 1

for index, answer_id in enumerate(answer_ids):
    if answer_id not in answer_id_count.keys():
        answer_id_count[answer_id] = 0

    if answer_id_count[answer_id] < sample_limit_count:
        '''
            データをリストに格納
        '''
        selected_answer_ids.append('%04d_%02d' % (answer_id, answer_id_count[answer_id]))
        selected_questions.append(questions[index])
        selected_separated_sentences.append(_separated_sentences[index])
        answer_id_count[answer_id] += 1

### (2-2) コーパスにタグ付け

models.doc2vecの仕様に従います。

In [7]:
from gensim import models
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

def doc_to_sentence(sentences, name):
    words = sentences.split(' ')
    return TaggedDocument(words=words, tags=[name])

def corpus_to_sentences(separated_sentences, answer_ids):
    for idx, (doc, name) in enumerate(zip(separated_sentences, answer_ids)):
        yield doc_to_sentence(doc, name)

### (2-3) 学習処理／モデルのシリアライズ

In [8]:
'''
    ユニークになったコーパス／ラベルから、
    学習セットを生成する
    
    教師データ件数＝173
'''
sentences = corpus_to_sentences(selected_separated_sentences, selected_answer_ids)
sentence_list = list(sentences)
len(sentence_list)

173

In [9]:
'''
    パラメータ変更点：
    　学習時にDBOWを使用する
    　featureの数をサンプル件数に近い値に設定
    　反復回数をfeature数の10倍に設定
'''
n_feature = round(len(sentence_list) + 49, -2) # 切り上げし、100の倍数に変更
n_iter = n_feature * 10
print('Doc2Vec: feature(size)=%d, iter=%d' % (n_feature, n_iter))

model = Doc2Vec(dm=0, size=n_feature, min_count=1, iter=n_iter)

Doc2Vec: feature(size)=200, iter=2000


In [10]:
'''
    ボキャブラリ生成／学習実行
    学習モデルは、ファイルに保存しておく
'''
model.build_vocab(sentence_list)
model.train(sentence_list)

model_path = 'prototype/better_algorithm/doc2vec.model'
model.save(model_path)

In [11]:
'''
    モデル内に保持されているベクトルの数を取得
    （教師データ数と同じであることを確認）
'''
len(model.docvecs)

173

In [12]:
model.docvecs

### (2-4) 予測処理

質問文は一部「揺れ」を与えたものとなっております

In [13]:
model_path = 'prototype/better_algorithm/doc2vec.model'

def predict(word, model_path):
    '''
        予測処理にかけるコーパスを生成
        （学習セット作成時と同じ関数を使用）
    '''
    corpus = Nlang_naive.split(word).split()

    '''
        コーパスからベクトルを生成し、
        ロードしたモデルから類似ベクトルを検索
    '''
    loaded_model = models.Doc2Vec.load(model_path)
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    return corpus, ret

In [14]:
'''
    マウスが破損（正解＝4458）
'''
predict('マウスが破損', model_path)

(['マウス', 'が', '破損'],
 [('4458_00', 0.8074274063110352),
  ('4459_00', 0.7710082530975342),
  ('4530_00', 0.6882411241531372),
  ('4514_00', 0.6427896022796631),
  ('4480_00', 0.6369040012359619),
  ('4526_00', 0.6365129947662354),
  ('4495_00', 0.6246248483657837),
  ('4513_00', 0.6231459379196167),
  ('4508_00', 0.5898932814598083),
  ('4499_00', 0.5892723798751831)])

In [15]:
'''
    無線を使用したい（正解＝4516）
'''
predict('無線を使用したい', model_path)

(['無線', 'を', '使用', 'する', 'たい'],
 [('4516_00', 0.8555318117141724),
  ('4521_00', 0.7502171993255615),
  ('4592_00', 0.7111111879348755),
  ('4577_00', 0.6952493786811829),
  ('4494_00', 0.6854709386825562),
  ('4447_00', 0.6571743488311768),
  ('4429_00', 0.652779221534729),
  ('4595_00', 0.6526064276695251),
  ('4493_00', 0.6501202583312988),
  ('4562_00', 0.6396880149841309)])

In [16]:
'''
    情報システムのアドレス（正解＝7040）
'''
predict('情報システムのアドレス', model_path)

(['情報', 'システム', 'の', 'アドレス'],
 [('7065_00', 0.7692917585372925),
  ('7040_00', 0.7492478489875793),
  ('4513_00', 0.6830494999885559),
  ('7064_00', 0.6686514616012573),
  ('4588_00', 0.6423662900924683),
  ('4593_00', 0.6389544010162354),
  ('4495_00', 0.6163446307182312),
  ('4600_00', 0.6045581102371216),
  ('4500_00', 0.5898104906082153),
  ('4460_00', 0.5889899134635925)])

In [17]:
'''
    Office2010を使用したいのですが（正解＝4625）
'''
predict('Office2010を使用したいのですが', model_path)

(['Ｏｆｆｉｃｅ', '２０１０', 'を', '使用', 'する', 'たい', 'の', 'です', 'が'],
 [('4577_00', 0.7587103843688965),
  ('4625_00', 0.718859076499939),
  ('4592_00', 0.7107665538787842),
  ('4429_00', 0.6994318962097168),
  ('4489_00', 0.6975282430648804),
  ('4516_00', 0.6899634599685669),
  ('4558_00', 0.6529085636138916),
  ('4521_00', 0.6428132057189941),
  ('4623_00', 0.6356406211853027),
  ('7065_00', 0.6308330297470093)])

In [18]:
'''
    携帯からサイボウズを使いたいのですが、どうしたら出来ますか？
    （正解＝4504だが、学習時の教師データから抜け落ちたもの）
'''
predict('携帯からサイボウズを使いたいのですが、どうしたら出来ますか？', model_path)

(['携帯',
  'から',
  'サイボウズ',
  'を',
  '使う',
  'たい',
  'の',
  'です',
  'が',
  'どう',
  'する',
  'た',
  '出来る',
  'ます',
  'か'],
 [('4590_00', 0.6622971892356873),
  ('4501_00', 0.6620618104934692),
  ('4468_00', 0.6150730848312378),
  ('7056_00', 0.6082926988601685),
  ('4472_00', 0.594771683216095),
  ('4541_00', 0.5945086479187012),
  ('4473_00', 0.5840214490890503),
  ('4560_00', 0.5561652183532715),
  ('4497_00', 0.5530551671981812),
  ('4478_00', 0.5397164821624756)])

## (3) accuracy 測定

ここからが本題です

In [19]:
model_path = 'prototype/better_algorithm/doc2vec.model'

len(selected_separated_sentences)

173

In [20]:
def predict_similarity(separated_sentence, model_path):
    corpus = separated_sentence.split()
    loaded_model = models.Doc2Vec.load(model_path)
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    answer_id, similarity = ret[0]
    return corpus, answer_id, similarity

In [21]:
def get_prediction_statistics(separated_sentences, answer_ids, model_path):
    '''
        学習セットの質問文をそのまま予測処理にかけて、
        回答を予測
    '''
    statistics = []
    for i, _ in enumerate(separated_sentences):
        sentence = separated_sentences[i]
        preferred_answer_id = answer_ids[i]
        corpus, answer_id, similarity = predict_similarity(separated_sentences[i], model_path)
        corpus_len = len(corpus)
        statistics.append((i, corpus_len, preferred_answer_id, answer_id, similarity))

    return statistics

In [22]:
prediction_statistics = get_prediction_statistics(selected_separated_sentences, selected_answer_ids, model_path)

In [23]:
ncorrect_by_corpus_len = {}
nsample_by_corpus_len = {}

ncorrect = 0
nsample = 0

'''
    予測結果を、質問文の単語数毎／回答ID毎に統計する
'''
for statistics in prediction_statistics:
    i, corpus_len, preferred_answer_id, answer_id, similarity = statistics
    
    '''
        質問文の単語数ごとに統計を取る
    '''
    if corpus_len not in nsample_by_corpus_len.keys():
        ncorrect_by_corpus_len[corpus_len] = 0
        nsample_by_corpus_len[corpus_len] = 0
    nsample_by_corpus_len[corpus_len] += 1
    
    '''
        正解かどうか検査
        （NNNN_nn 形式ラベルの上４桁が一致していれば正解とします）
    '''
    nsample += 1
    if preferred_answer_id[0:4] == answer_id[0:4]:
        ncorrect += 1
        ncorrect_by_corpus_len[corpus_len] += 1

In [24]:
'''
    質問文の単語数ごとの統計情報を編集
'''
info_by_corpus_len = []
for k, v in ncorrect_by_corpus_len.items():
    info_by_corpus_len.append((
        k, 
        ncorrect_by_corpus_len[k]/nsample_by_corpus_len[k], 
        ncorrect_by_corpus_len[k], 
        nsample_by_corpus_len[k]
    ))

In [25]:
'''
    全体の正解率
'''
print("accuracy=%0.3f (%d/%d)" % (
    ncorrect/nsample, ncorrect, nsample))

accuracy=0.965 (167/173)


In [26]:
'''
    質問文の単語数ごとの正解率をリスト
'''
for info in info_by_corpus_len:
    print("word_count=%2d: accuracy=%0.3f (%d/%d)" % (
        info[0], info[1], info[2], info[3]
    ))

word_count= 1: accuracy=0.000 (0/1)
word_count= 2: accuracy=1.000 (7/7)
word_count= 3: accuracy=1.000 (6/6)
word_count= 4: accuracy=1.000 (10/10)
word_count= 5: accuracy=0.889 (16/18)
word_count= 6: accuracy=0.905 (19/21)
word_count= 7: accuracy=0.950 (19/20)
word_count= 8: accuracy=1.000 (20/20)
word_count= 9: accuracy=1.000 (12/12)
word_count=10: accuracy=1.000 (7/7)
word_count=11: accuracy=1.000 (10/10)
word_count=12: accuracy=1.000 (11/11)
word_count=13: accuracy=1.000 (11/11)
word_count=14: accuracy=1.000 (5/5)
word_count=15: accuracy=1.000 (6/6)
word_count=16: accuracy=1.000 (2/2)
word_count=17: accuracy=1.000 (2/2)
word_count=19: accuracy=1.000 (1/1)
word_count=20: accuracy=1.000 (1/1)
word_count=22: accuracy=1.000 (1/1)
word_count=30: accuracy=1.000 (1/1)
